In [ ]:
import numpy as np
import pandas as pd
import json
from datetime import datetime 

import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials	

In [ ]:
class Message:
  def __init__(self):
    self.__content = dict()

  def __make_tuple(self, elem):
    return elem if type(elem) == tuple else tuple(elem) if type(elem) == list else tuple([elem])

  def add_request(self, args):
    self.__content["request"] = dict(filter(lambda x: x, map(lambda x: (x[0], self.__make_tuple(x[1])) if x[1] else (), args.items())))
    
  def add_log(self, logtext):
    self.__content["log"] = logtext

  def get_content(self):
    return self.__content

  def get_json(self):
    return json.dumps(self.__content, ensure_ascii=False)


In [ ]:
class SheetAdress():
  def __init__(self, sheetname, sheetrange):
    self.__sheetname = sheetname
    self.__sheetrange = sheetrange
  
  def n(self):
    return self.__sheetname

  def r(self):
    return self.__sheetrange

  def a(self):
    return (self.__sheetname, self.__sheetrange)

In [ ]:
class Controller:
  def __init__(self):
    self.__parserGlobal = ParserGoogleSheet("some url") 
    self.__engine = Engine(self.__parserGlobal)

  def schedule_request(self, group_id = (), course = (), teacher_id = (), class_id = (), weekday_id = (), pair_numbers = ()):
    #typle args are expected
    msg = Message()
    msg.add_request({"groups": group_id,  "courses": course, "teachers": teacher_id, "classes": class_id, "weekdays": weekday_id, "pair_numbers": pair_numbers})
    return self.__engine.schedule_request(msg)

  def schedule_db_request(self):
    return self.__engine.schedule_db_request()
  


In [ ]:
class Engine:
  def __init__(self, parserGlobal):
    self.__scheduleData = DataFrame(parserGlobal)

  def schedule_request(self, msg):
    return self.__scheduleData.request(msg)
  
  def schedule_db_request(self):
      return self.__scheduleData.get_tables()

In [ ]:
class Parser:
  def __init__(self, res):
    self._resource = res

  def parse(self):
    pass

1 строчка -- название курса

2 строчка -- пустая

3 строчка -- названия групп

4 строчка -- профили групп через ';'

5 сторочка -- начинается расписание

In [ ]:
class ParserTables(Parser):
  def parse(self, msg):
    for i in self._resource:
      if i._get_name() == "pairse":
        res = self._resource
    req = msg.get_content()["request"]

    if req["courses"]:
      req["groups"] = req.get("groups", ())+tuple((self._resource["groups"][(self._resource["groups"]["Course"].isin(req["courses"]))].index))
      req.pop("courses")

    for param in req:
      if param != "courses":
        res = res[(res[param].isin(req[param]))]    
    return res

In [ ]:
class ParserTeacherTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS

  def __teacher_table_parser(self, sheetAddress):
      ans = self.__parserGS.get_sheet(sheetAddress)
      df = pd.DataFrame(ans[1:])
      df.columns = ans[0]
      df["Преподаватель"] = df["Преподаватель"].apply(self.__expand_line(3, "-"))
      df["Должность"] = df["Должность"].apply(self.__expand_line(1, "-"))
      teachers = df[["Преподаватель", "Должность"]]
      return teachers

  def __expand_line(self, n, symb):
      symb = " " + symb
      return (lambda s: s + symb*(n - len(s.split())) if s else symb*n)

  def parse(self):
    sheet_values = pd.concat([
        self.__teacher_table_parser(SheetAdress("Список преподавателей институских кафедр", "A1:C100")), 
        self.__teacher_table_parser(SheetAdress("Список преподавателей баз.кафедр", "A1:D200"))])
    teachers = Table("teachers", ['Surname','Name','Lastname','Post'])
    
    for i in range(len(sheet_values)):
      teachers.add_line(sheet_values.iloc[i]["Преподаватель"].split()+[sheet_values.iloc[i]["Должность"].lower()])
    
    teachers.sort_by("Surname")
    teachers.reset_index()
    return teachers

In [ ]:
class ParserClassTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS
  
  def parse(self):
    sheet_values = self.__parserGS.get_sheet(SheetAdress("аудиторный фонд", "A1:AA2"))
    classes = list(filter(lambda x: str(x).isdigit(), sheet_values[1]))
    return Table("classes", ["Number"], [classes])

In [ ]:
class ParserGroupTable(Parser):
  def __init__(self, parserGS):
    super().__init__(None)
    self.__parserGS = parserGS

  def parse(self):
    groups = Table("groups", ['Course', 'Group'])
    for num in range(1, 7):
      sheet_values = self.__parserGS.get_sheet(SheetAdress(str(num)+" курс ", "A1:P60"))
      group_names = list(filter(lambda x: x and not str(x).isalpha(), sheet_values[2]))
      groups.add_lines([[num]*len(group_names), group_names]) #(cource, group_name)
    return groups

In [ ]:
class DataFrame:
  def __init__(self, resource, names = [], tables = []):
    self.__tables = []
    self.__parserGS = ParserGoogleSheet(resource)
    
    self.__update()

    self.__parserLocal = ParserTables(self.__tables)
    self.__last_update = datetime.now()

  def __update(self):
    self.__tables = self.__parserGS.parse()

  def __check_update(self):
    update_time = 2*60*60
    if (datetime.now() - self.__last_update).seconds > update_time:
      self.__update()
    return True

  def add(self, name, table):
    if name in self.__tables.keys():
      raise Exception("this name already exists")
    self.__tables[name] = table
  
  def get_tables(self):
    return self.__tables

  def request(self, msg):
    self.__check_update()
    return self.__parserLocal.parse(msg)
  





  def print_schedule_tables(self): #to del
    self.__parserGS.print_schedule_tables()

  def print_tables(self): #to del
    for i in self.__tables:
      print(i.get_table())

  def f(self):
    return self.__tables[-1]["Суббота"]
  
  

In [ ]:
class Table:
  def __init__(self, name, colomns, lines=[]):
    self.__name = name
    self.__colomns = colomns
    if not lines:
      lines = [pd.Series()]*len(colomns)
    self.data = pd.DataFrame(self.__make_dir(colomns, lines))

  def __make_dir(self, keys, values):
    return {k:v for k, v in zip(keys, values)}

  def add_line(self, line):
    self.data = self.data.append(self.__make_dir(self.data.columns, line), ignore_index=True)
  
  def add_lines(self, colomns):
    for line in zip(*colomns):
      self.add_line(line)

  def get_table(self):
    return self.data

  def get_name(self):
    return self.__name

  def sort_by(self, param):
    self.data = self.data.sort_values(by=param)
  
  def reset_index(self):
    self.data = self.data.reset_index()

  def __getitem__(self, item):
    for c in self.__colomns:
      if item in list(self.data[c]):
        return  self.data[self.data[c] == item]

In [ ]:
class ParserGoogleSheet(Parser):
  def __init__(self, res):
    super().__init__(res)
    self.__init_service_acc()
    self.__tables = []

  def __init_service_acc(self):
    CREDENTIALS_FILE = 'central-diode-342919-c35aafd1b173.json'
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        CREDENTIALS_FILE, 
        ['https://www.googleapis.com/auth/spreadsheets', 
         'https://www.googleapis.com/auth/drive'])


    httpAuth = credentials.authorize(httplib2.Http()) 
    self.service = apiclient.discovery.build('sheets', 'v4', http = httpAuth)

  def get_sheet(self, sheetAddress): #сделать обёртку для sheet_name, range
    ranges = [sheetAddress.n()+"!"+sheetAddress.r()]
    results = self.service.spreadsheets().values().batchGet(spreadsheetId = "1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc", #self._resource, 
                                        ranges = ranges, 
                                        valueRenderOption = 'UNFORMATTED_VALUE',  
                                        dateTimeRenderOption = 'FORMATTED_STRING').execute() 
    return results['valueRanges'][0]['values']

  def __getitem__(self, item):
    for t in self.__tables:
      if t.get_name() == item:
        return t

  def parse(self):
    self.__tables.append(ParserTeacherTable(self).parse())
    self.__tables.append(ParserGroupTable(self).parse())
    self.__tables.append(ParserClassTable(self).parse())
    self.__tables.append(Table("weekdays", ["Weekday"], [["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота"]]))
    self.__tables.append(Table("pairs", ["Time"], [["9:00-10:25", "10:35-12:00", "12:10-13:35", "13:45-15:10", "15:20-16:45", "16:55-18:20", "18:25-19:45"]]))
    return self.__tables
    
  def print_schedule_tables(self): #to del
    for num in range(1, 7):
      sheet_values = self.get_sheet(SheetAdress(str(num)+" курс ", "A1:P60"))
      #таблицы с 4го курсапарсятся плохо из-за их форматирования
      if num <= 3:
        prev_line = []
        for line in sheet_values[4:]:
          if line:
            line += [line[-1]]*(7-len(line))
            for cell_id in range(len(line)-1):
              if not line[cell_id]:
                line[cell_id] = prev_line[cell_id]
            prev_line = line
          print(line) 
          if line:
            print(self["weekdays"][line[0]].index[0])
            print(self["pairs"][line[1]].index[0])
            if len(line[2].split("/")) == 3:
              class_ = ''.join(line[2].split("/")[2].split())
              if class_.isdigit():
                print(self["classes"][int(class_)].index[0])
            else:
              print(line[2].split("/"))
          print("-------")


  def dprint_schedule_tables(self): #to del
    for num in range(1, 7):
      sheet_values = self.get_sheet(SheetAdress(str(num)+" курс ", "A1:P60"))
      #таблицы с 4го курсапарсятся плохо из-за их форматирования
      if num <= 3:
        prev_line = []
        for line in sheet_values[4:]:
          if line:
            line += [line[-1]]*(7-len(line))
            for cell_id in range(len(line)-1):
              if not line[cell_id]:
                line[cell_id] = prev_line[cell_id]
            prev_line = line
          print(line)


## tests

In [ ]:
tf = DataFrame("1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc")
tf.print_tables()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


     index    Surname        Name       Lastname                   Post
0       61     Авдеев       Антон        Юрьевич          преподаватель
1       62  Аврутский    Всеволод       Игоревич              ассистент
2        0  Аврутский    Всеволод       Игоревич              ассистент
3       63     Алиева       Диана  Александровна          преподаватель
4       64     Анохин       Игорь  Александрович          преподаватель
..     ...        ...         ...            ...                    ...
183     59     Щеглов      Андрей      Сергеевич              ассистент
184    185   Щербаков    Владимир     Николаевич          преподаватель
185    186    Щербань  Константин     Степанович              профессор
186     60       Юдин      Михаил  Александрович  старший преподаватель
187    187       Юдин     Василий     Николаевич              профессор

[188 rows x 5 columns]
    Course    Group
0      1.0  Б03-111
1      1.0  Б03-112
2      1.0  Б03-113
3      1.0  Б03-114
4      1.0  

In [ ]:
tf.print_schedule_tables()

['Понедельник', '9:00-10:25', ' Дискретная математика   /  Здор А. Г./ 306', ' Дискретная математика   /  Здор А. Г./ 306', ' Дискретная математика   /  Здор А. Г./ 306', ' Дискретная математика   /  Здор А. Г./ 306', ' Дискретная математика   /  Здор А. Г./ 306']
0
0
1
-------
['Понедельник', '10:35-12:00', 'Многомерный анализ, интегралы и ряды /  Ковалев В. П. /  306', 'Многомерный анализ, интегралы и ряды /  Ковалев В. П. /  306', 'Многомерный анализ, интегралы и ряды /  Ковалев В. П. /  306', 'Многомерный анализ, интегралы и ряды /  Ковалев В. П. /  306', 'Многомерный анализ, интегралы и ряды /  Ковалев В. П. /  306']
0
1
1
-------
['Понедельник', '12:10-13:35', 'Информатика (Архитектура компьютеров и операционные системы) /Матяш Е. С./ 250', 'Информатика (Архитектура компьютеров и операционные системы) /Матяш Е. С./ 250', 'Информатика (Архитектура компьютеров и операционные системы) /Матяш Е. С./ 250', 'Информатика (Архитектура компьютеров и операционные системы) /Матяш Е. С./ 250

In [ ]:
class Controller:
  def __init__(self):
    self.__parserGlobal = ParserGoogleSheet("some url") 
    self.__engine = Engine(self.__parserGlobal)

  def schedule_request(self, group_id = (), course = (), teacher_id = (), class_id = (), weekday_id = (), pair_numbers = ()):
    #typle args are expected
    msg = Message()
    msg.add_request({"groups": group_id,  "courses": course, "teachers": teacher_id, "classes": class_id, "weekdays": weekday_id, "pair_numbers": pair_numbers})
    return self.__engine.schedule_request(msg)

  def schedule_db_request(self):
    return self.__engine.schedule_db_request()

In [ ]:
contr = Controller()
contr.schedule_request(course=1, weekday_id=[1, 2])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


TypeError: ignored

In [ ]:
contr.schedule_db_request()["classes"]

In [ ]:
df = pd.DataFrame({"a": pd.Series(1), "b": pd.Series(2)})
df = df.append({"a" : 3, "b": 4}, ignore_index=True)
df

## api zone

адрес сервисного аккаунта - 
service-admin-230@central-diode-342919.iam.gserviceaccount.com 	


Лимиты Google Sheets API

Google почти ко всем своим API доступ выдает БЕСПЛАТНО и без премодерации (в тестовом режиме). Однако ограничивает. К некоторым апи ограничения сильные, а к Sheets API вполне приемлемые.

    Read requests per minute 300
    Read requests per minute per user 60
    Read requests per day Unlimited

    Write requests per minute 300
    Write requests per minute per user 60
    Write requests per day Unlimited 

In [ ]:
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials	

CREDENTIALS_FILE = 'central-diode-342919-c35aafd1b173.json'  # Имя файла с закрытым ключом, вы должны подставить свое

# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API 
'''
spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Первый тестовый документ', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Лист номер один',
                               'gridProperties': {'rowCount': 100, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId'] # сохраняем идентификатор файла
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)'''

file_id = '1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc'
request = service.files().export_media(fileId=file_id,
                                             mimeType='application/pdf')
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))



## gs interaction info

In [ ]:
#создание таблички
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'kokhanova.va@phystech.edu'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

In [ ]:
#добавление листа
results = service.spreadsheets().batchUpdate(
    spreadsheetId = spreadsheetId,
    body = 
{
  "requests": [
    {
      "addSheet": {
        "properties": {
          "title": "Еще один лист",
          "gridProperties": {
            "rowCount": 20,
            "columnCount": 12
          }
        }
      }
    }
  ]
}).execute()


# Получаем список листов, их Id и название
spreadsheet = service.spreadsheets().get(spreadsheetId = spreadsheetId).execute()
sheetList = spreadsheet.get('sheets')
for sheet in sheetList:
    print(sheet['properties']['sheetId'], sheet['properties']['title'])
    
sheetId = sheetList[0]['properties']['sheetId']

print('Мы будем использовать лист с Id = ', sheetId)

In [ ]:
#чтение
ranges = ["1 курс !A1:G51"] # 
          
results = service.spreadsheets().values().batchGet(spreadsheetId = "1s_u2pPZ3xdu_tBrVy7hriV2xj15OP9evJfVAuzFyZSc", 
                                     ranges = ranges, 
                                     valueRenderOption = 'UNFORMATTED_VALUE',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
sheet_values = results['valueRanges'][0]['values']

first_pair = '9:00-10:25'

for i in range(len(sheet_values)):
  if any(map(lambda x: x == "Понедельник", sheet_values[i])):
    start_line_id = i

prev_line = []

for line in sheet_values[start_line_id:]:
  if line:
    line += [line[-1]]*(7-len(line))
    for cell_id in range(len(line)-1):
      if not line[cell_id]:
        line[cell_id] = prev_line[cell_id]
    prev_line = line
  print(line)



In [ ]:
some_line